# LME and Creatinine

## Joining Data

In [1]:
library(data.table)
library(nlme)

creatinine.data <- data.table(read.csv('creat.csv'), key='group')
head(creatinine.data)

id,group,age,scr
1,1,35.765,0.182
1,1,37.990,0.088
3,1,51.083,0.156
3,1,52.386,0.116
3,1,52.805,0.087
3,1,52.997,0.067


In [2]:
group.data <- data.table(
    group=c(1,2,3,4),
    kidney.disease=c(1,1,0,0),
    hypertension=c(1,0,1,0),
    key='group')
group.data

group,kidney.disease,hypertension
1,1,1
2,1,0
3,0,1
4,0,0


In [3]:
creatinine.group.data <- creatinine.data[group.data]
setkey(creatinine.group.data, id)
head(creatinine.group.data)

id,group,age,scr,kidney.disease,hypertension
1,1,35.765,0.182,1,1
1,1,37.990,0.088,1,1
2,2,24.997,1.429,1,0
2,2,27.441,1.111,1,0
2,2,30.524,1.429,1,0
3,1,51.083,0.156,1,1


## Fitting LME Model

In [4]:
scr.model <- lme(scr ~ age*kidney.disease,
                 random=reStruct(~age|id, pdClass='pdDiag'),
                 correlation=corExp(form=~age|id, nugget=TRUE),
                 method='ML', data=creatinine.group.data)
summary(scr.model, adjustSigma=FALSE)

Linear mixed-effects model fit by maximum likelihood
 Data: creatinine.group.data 
        AIC       BIC   logLik
  -127.9923 -79.67728 72.99617

Random effects:
 Formula: ~age | id
 Structure: Diagonal
        (Intercept)         age  Residual
StdDev:   0.1317441 0.004823218 0.2258346

Correlation Structure: Exponential spatial correlation
 Formula: ~age | id 
 Parameter estimate(s):
    range    nugget 
4.6701089 0.2299759 
Fixed effects: scr ~ age * kidney.disease 
                        Value  Std.Error  DF   t-value p-value
(Intercept)         1.1906761 0.05423195 964 21.955252  0.0000
age                -0.0031083 0.00143568 964 -2.165012  0.0306
kidney.disease      0.3135749 0.07137339 617  4.393443  0.0000
age:kidney.disease -0.0166492 0.00185709 964 -8.965191  0.0000
 Correlation: 
                   (Intr) age    kdny.d
age                -0.928              
kidney.disease     -0.760  0.705       
age:kidney.disease  0.718 -0.773 -0.935

Standardized Within-Group Residuals:

## Covariance for Subject $i$

In [5]:
make.covariance <- function(model, i) {
    # Error term, usually denoted epsilon
    error.correlation <- corMatrix(model$modelStruct$corStruct)[[as.character(i)]]
    if (is.null(error.correlation)) { error.correlation <- 1 }
    # Random effects correlation, Z * G * tranpose(Z).
    Z <- cbind(1, model$data[J(i), age])
    random.correlation <- Z %*% as.matrix(model$modelStruct$reStruct$id) %*% t(Z)
    # Convert correlation matrix into covariance matrix.
    (random.correlation + error.correlation)*(model$sigma*model$sigma)
}

## $\beta_j$ Covariance 

### Maximum Likelihood Estimate

This assumes that the random effects model is correct. I can also be used as the *bread* part of the sandwich estimator.

In [6]:
make.ml.parameter.covariance <- function(model) {
    groups <- unique(scr.model$groups)$id
    chol2inv(chol(Reduce(`+`, lapply(groups, function(i) {
        X <- as.matrix(cbind(
            `(intercept)`=1,
            model$data[J(i),
            list(age, kidney.disease, `age:kidney.disease`=age*kidney.disease)]))
        t(X) %*% chol2inv(chol(make.covariance(model, i))) %*% X
    }))))
}

#sqrt(diag(make.ml.parameter.covariance(scr.model)))